In [45]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [46]:
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [47]:
test_data.data.shape

(10000, 32, 32, 3)

In [8]:
'''
1. Torch tensor
train_target = torch.tensor(train['Target'].values.astype(np.float32)) # train ist ein DF
train = torch.tensor(train.drop('Target', axis = 1).values.astype(np.float32))

2. Torch dataset
train_tensor = data_utils.TensorDataset(train, train_target)
test_tensor = data_utils.TensorDataset(test, test_target)

3. Torch dataloader
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)
'''


"\n1. Torch tensor\ntrain_target = torch.tensor(train['Target'].values.astype(np.float32)) # train ist ein DF\ntrain = torch.tensor(train.drop('Target', axis = 1).values.astype(np.float32))\n\n2. Torch dataset\ntrain_tensor = data_utils.TensorDataset(train, train_target)\ntest_tensor = data_utils.TensorDataset(test, test_target)\n\n3. Torch dataloader\ntrain_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)\ntest_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)\n"

In [48]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


In [49]:
for X, y in train_dataloader:
    print(X.shape)
    break

torch.Size([64, 3, 32, 32])


In [50]:
# torch.full(size=(9,), fill_value=1)

In [51]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*32*32, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

Using cpu device


In [52]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [53]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss() # mean absolute error , mse

In [54]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [61]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad() # fit
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [62]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [63]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.124074  [    0/50000]
loss: 1.079955  [ 6400/50000]
loss: 0.904971  [12800/50000]
loss: 1.447746  [19200/50000]
loss: 1.251033  [25600/50000]
loss: 1.077151  [32000/50000]
loss: 0.990718  [38400/50000]
loss: 1.174186  [44800/50000]
Test Error: 
 Accuracy: 48.6%, Avg loss: 0.024499 

Epoch 2
-------------------------------
loss: 0.975505  [    0/50000]
loss: 1.089214  [ 6400/50000]
loss: 0.928060  [12800/50000]
loss: 1.397614  [19200/50000]
loss: 1.174275  [25600/50000]
loss: 1.197895  [32000/50000]
loss: 0.997655  [38400/50000]
loss: 1.229190  [44800/50000]
Test Error: 
 Accuracy: 47.7%, Avg loss: 0.025368 

Epoch 3
-------------------------------
loss: 0.992564  [    0/50000]
loss: 1.063164  [ 6400/50000]
loss: 0.910031  [12800/50000]
loss: 1.481214  [19200/50000]
loss: 1.152455  [25600/50000]
loss: 1.223453  [32000/50000]
loss: 1.023475  [38400/50000]
loss: 1.204797  [44800/50000]
Test Error: 
 Accuracy: 47.1%, Avg loss: 0.025918 

Epoc

In [65]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [66]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [72]:
classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

model.eval()
x, y = test_data[0]  # returns (C,H,W) and label
with torch.no_grad():
    x = x.unsqueeze(0).to(device)  # add batch dimension -> shape [1, 3, 32, 32]
    pred = model(x)                # output shape [1, 10]
    predicted = classes[int(pred.argmax(1).item())]
    actual = classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "cat", Actual: "cat"


In [22]:
test_data.data.shape

torch.Size([10000, 28, 28])